# Защита персональных данных клиентов

Нужно защитить данные клиентов компании. Необходимо разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы. Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<ul><li><strong>Признаки:</strong> пол, возраст и зарплата застрахованного, количество членов его семьи.</li><li><strong>Целевой признак:</strong> количество страховых выплат клиенту за последние 5 лет.</li></ul>

In [1]:
try:
    %load_ext lab_black
except:
    "no lab_black"

## Импорты

!pip install pandas-profiling

In [2]:
import numpy as np
import pandas as pd
import matplotlib as plt

from pandas_profiling import ProfileReport
import sweetviz

from numpy.random import default_rng

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression

## Загрузка данных

Загрузим и изучим данные:

In [3]:
data = pd.read_csv("/datasets/insurance.csv")

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [5]:
data.head(10)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


In [6]:
X = data.drop("Страховые выплаты", axis=1)
y = data["Страховые выплаты"]
X.head(15)

,Пол,Возраст,Зарплата,Члены семьи
0,1,41.0,49600.0,1
1,0,46.0,38000.0,1
2,0,29.0,21000.0,0
3,0,21.0,41700.0,2
4,1,28.0,26100.0,0
5,1,43.0,41000.0,2
6,1,39.0,39700.0,2
7,1,25.0,38600.0,4
8,1,36.0,49700.0,1
9,1,32.0,51700.0,1


In [7]:
y.value_counts()

0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64

### Pandas profiling 

In [8]:
profile = ProfileReport(data, "All data profiling report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Видим, что 88.7% клиентов не запрашивало страховых выплат.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

- $w_c$ — вектор весов линейной регрессии для признаков умноженных на матрицу P (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$


Признаки умножают на обратимую матрицу. В таком случае качество линейной регрессии не изменится. Докажем это и найдем, как связаны параметры линейной регрессии в исходной задаче и в преобразованной.

### Получение формулы для весов линейной регрессии 

$$
MSE=\frac{1}{n} \sum_{i=1}^{i=n} e_i^2 
$$

$$
\begin{aligned}
 \sum_{i=1}^{i=n} e_i^2=e_1^2+e_2^2+e_3^2+e_4^2+\ldots+e_n^2 =
\end{aligned}
$$

$$
 = [e_1 e_2 e_3 e_4 \ldots e_n]^T x [e_1 e_2 e_3 e_4 \ldots e_n]
$$

$$
MSE=\frac{1}{n}(y-Xw)^T(y-Xw) =
$$

зная, что $(AB)^T=B^T A^T$ и $(A - B)^T=A^T - B^T$, получаем:

$$
MSE=\frac{1}{n}(y^T y - w^T X^T y- y^T X w + w^T X^T Xw) 
$$

Так как $w^T X^T y=(y^T X w)^T$ и т.к. это матрица 1x1: 

$$
(1x5000) @ (5000x4) @ (4x1) = (1x1)
$$

То транспонированная матрица равна нетранспонированной, таким образом мы можем заменить $y^T X w$ на $w^T X^T y$ и получаем финальное выражение для $MSE$:

$$
MSE=\frac{1}{n}(y^T y - 2 w^T X^T y + w^T X^T Xw) 
$$

Далее чтобы минимизировать данную функцию нужно взять её градиент:

$$
\Delta MSE=\frac{1}{n}(\Delta y^T y - 2 \Delta w^T X^T y + \Delta w^T X^T Xw) 
$$

используя следующие правила дифференциирования матриц продифференцируем по $ w $ :

* Если $ F=w^T A w $ ,то $ \frac{\delta F}{\delta w} = 2Aw $
* Если $ F=w^T A $ ,то $ \frac{\delta F}{\delta w} = A $
$$
\Delta MSE=\frac{1}{n}(0 - 2 X^T y + 2 X^T X w) 
$$

$$
=\frac{2}{n}(X^T X w-  X^T y) 
$$

Приравнивая это выражение нулю находим экстремум (минимум):

$$
\frac{2}{n}(X^T X w-  X^T y)=0
$$

$$
X^T X w=X^T y
$$

Откуда и получаем формулу обучения:

$$
w = (X^T X)^{-1} X^T y
$$


### Получение формулы для весов линейной регрессии после преобразования 

Проделаем аналогичные действия в случае матрицы признаков $X$ умноженной на $P$ (то есть применим линейное преобразование P к признакам X). $MSE_c$ - это показатель среднеквадратичнго отклонения для новой матрицы признаков $X_c=X P$

$$
MSE_c=\frac{1}{n} \sum_{i=1}^{i=n} e_i^2 
$$

$$
\begin{aligned}
\sum_{i=1}^{i=n} e_i^2=e_1^2+e_2^2+e_3^2+e_4^2+\ldots+e_n^2 =
\end{aligned}
$$

$$
 = [e_1 e_2 e_3 e_4 \ldots e_n]^T x [e_1 e_2 e_3 e_4 \ldots e_n]
$$

$$
MSE_c=\frac{1}{n}(y-XPw_c)^T(y-XPw_c) =
$$

зная, что $(AB)^T=B^T A^T$ и $(A - B)^T=A^T - B^T$, получаем:

$$
MSE_c=\frac{1}{n}(y^T y - w_c^T P^T X^T y- y^T X P w_c + w_c^T P^T X^T X P w_c) 
$$

Так как $w_c^T P^T X^T  y=(y^T P X w_c)^T$ и т.к. это матрица 1x1: 

$$
(1x5000) @ (5000x4)  @ (4x4) @ (4x1) = (1x1)
$$

То транспонированная матрица равна нетранспонированной, таким образом мы можем заменить $y^T X P w_c$ на $w_c^T P^T X^T y$ и получаем финальное выражение для $MSE_c$:

$$
MSE_c=\frac{1}{n}(y^T y - 2 w_c^T P^T X^T y + w_c^T P^T X^T X P w_c) 
$$

Далее чтобы минимизировать данную функцию нужно взять её градиент:

$$
\Delta MSE_c=\frac{1}{n}(\Delta y^T y - 2 \Delta w_c^T P^T X^T y + \Delta w_c^T P^T X^T XPw_c) 
$$

используя следующие правила дифференциирования матриц продифференцируем по $ w_c $ :

* Если $ F=w_c^T A w_c $ ,то $ \frac{\delta F}{\delta w_c} = 2Aw_c $ для симметричной $A$
* Если $ F=w_c^T A $ ,то $ \frac{\delta F}{\delta w_c} = A $
$$
\Delta MSE_c=\frac{1}{n}(0 - 2 P^T X^T y + 2 P^T X^T X P w_c) 
$$

$$
=\frac{2}{n}(P^TX^T X P w_c-  P^TX^T y) 
$$

Приравнивая это выражение нулю находим экстремум (минимум):

$$
\frac{2}{n}(P^TX^T X P w_c-  P^TX^T y) =0
$$

$$
P^TX^T X P w_c=P^TX^T y
$$

Откуда получаем формулу для $w_c$:

$$
w_c= (P^TX^T XP)^{-1}P^T X^T y
$$


Или можно ее записать так:

$$
w_c = ((XP)^T XP)^{-1}(XP)^T y
$$

Вернемся к предыдущему выражению, а именно:

$$
w_c= (P^TX^T XP)^{-1}P^T X^T y
$$

И подставим в него выражение из предыдущего пункта c обычным $w$, чтобы найти связь между $w_c$ и $w$:

$$
X^T X w=X^T y
$$

Вместо $X^T y$ подставляем $X^T X w$:

$$
w_c= (P^TX^T XP)^{-1}P^T X^T X w
$$

или

$$
w_c= (P^T(X^T XP))^{-1}P^T X^T X w
$$


далее используя свойства обратных матриц, а именно, что $(A^{-1})^T=(A^T)^{-1}$, $(A B)^T=B^T A^T$ и  $A^{-1} A=A  A^{-1}=E$ преобразуем к:

$$
w_c= (X^T XP)^{-1}(P^T)^{-1}P^T X^T X w
$$

$$
w_c= (X^T XP)^{-1}X^T X w
$$

$$
w_c= P^{-1}(X^T X)^{-1}(X^T X) w
$$

$$
w_c= P^{-1}w
$$

Подставив это в формулу для $MSE_c$
$$
MSE_c=\frac{1}{n}(y-XPw_c)^T(y-XPw_c) 
$$

Получаем:
$$
MSE_c=\frac{1}{n}(y-XPP^{-1}w)^T(y-XPP^{-1}w) 
$$
И получаем, что:
$$
MSE_c=MSE =\frac{1}{n}(y-Xw)^T(y-Xw) 
$$



**Ответ:** Качество линейной регрессии (MSE) не изменится при умножении признаков на обратимую матрицу.

**Обоснование:** Так как, в таком случае среднеквадратичное отклонение (MSE) ошибки для преобразованной матрицы равно среднеквадратичному отклонению (MSE) ошибки для исходной матрицы, что мы показали выше.

## Алгоритм преобразования

**Алгоритм**

Создадим матрицу $P$ (на которую будем умножать признаки) из случайных вещественных чисел в диапазоне [0.0, 1.0), назовем её `encryption_matrix`:

In [9]:
rng = default_rng(5_12_22)

In [10]:
encryption_matrix = rng.random((X.shape[1], X.shape[1]))
encryption_matrix

array([[0.21974032, 0.51943398, 0.50929578, 0.16595714],
       [0.73401819, 0.20145713, 0.88959769, 0.08139564],
       [0.82854767, 0.31178531, 0.97267918, 0.87623969],
       [0.109165  , 0.44105726, 0.5550065 , 0.60649281]])

Проверим что матрица обратима, то есть ее определитель не равен нулю:

In [11]:
print("det(P)=", np.linalg.det(encryption_matrix))

det(P)= 0.0644257343269832


В `X_c` запишем резкльтат умножения признаков на матрицу $P$:

In [12]:
X_c = X @ encryption_matrix
X_c

,0,1,2,3
0,41126.388189,15473.771703,48282.425209,43465.598129
1,31518.685542,11857.549937,37003.285397,33301.458785
2,17420.787642,6553.333806,20452.061144,18403.393893
3,34566.070640,13006.560220,40580.513432,36542.117228
4,21645.866492,8143.756874,25412.344668,22872.300857
...,...,...,...,...
4995,29599.922734,11137.258549,34750.665527,31285.248877
4996,43440.963803,16344.840943,50999.190438,45918.333525
4997,28102.664779,10574.433330,32992.726219,29707.366276
4998,27110.204514,10201.654362,31828.354699,28656.813893


Убедимся, еще раз, что наша матрица обратима, найдем обратную матрицу:

In [13]:
inverse_matrix = np.linalg.inv(encryption_matrix)
inverse_matrix

array([[ 2.72597509, -1.73625286,  2.58102027, -4.24187058],
       [ 3.99066664, -1.95184762,  0.96357009, -2.22216333],
       [-3.10228144,  3.09979741, -2.44602429,  3.96680601],
       [-0.55385187, -1.10470084,  1.07307576,  0.39829314]])

Убедимся, что детерминант обратной матрицы равен $\frac{1}{det(P)}$:

In [14]:
print("1/det(P)=", 1 / np.linalg.det(encryption_matrix))
print("det(inverse(P))=", np.linalg.det(inverse_matrix))

1/det(P)= 15.521747799173685
det(inverse(P))= 15.521747799173685


Декодируем значения признаков из `X_c` с помощью обратной матрицы:

In [15]:
X_c @ inverse_matrix

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-1.109554e-11,46.0,38000.0,1.000000e+00
2,1.782111e-13,29.0,21000.0,1.854575e-11
3,-2.599650e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,2.282261e-11
...,...,...,...,...
4995,-1.267589e-11,28.0,35700.0,2.000000e+00
4996,1.220474e-11,34.0,52400.0,1.000000e+00
4997,-2.139138e-11,20.0,33900.0,2.000000e+00
4998,1.000000e+00,22.0,32700.0,3.000000e+00


Видим, что получили исходные значения, но возникла погрешность из-за особенности операций с числами с плавающей запятой.

### Класс-шифровальщик

Напишем класс для шифрования численных признаков:

In [16]:
class NumericFeaturesEncrypter:
    def __init__(self):
        self.random_state = 5_12_22

    def transform(self, X, **transform_params):
        rng = default_rng(self.random_state)
        self.encryption_matrix = rng.random((self.matrix_size, self.matrix_size))
        return X @ self.encryption_matrix

    def get_encrypt_matrix(self):
        return self.encryption_matrix

    def get_decrypt_matrix(self):
        return np.linalg.inv(self.encryption_matrix)

    def fit(self, X, y=None, **fit_params):
        self.matrix_size = X.shape[1]
        return self

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X)
        return self.transform(X)

С помощью класса произведем все действия аналогично произведенным выше, убедимся, что получили те же результаты:

In [17]:
encryptor = NumericFeaturesEncrypter()

In [18]:
encryptor.fit(X)
encryptor.transform(X)

,0,1,2,3
0,41126.388189,15473.771703,48282.425209,43465.598129
1,31518.685542,11857.549937,37003.285397,33301.458785
2,17420.787642,6553.333806,20452.061144,18403.393893
3,34566.070640,13006.560220,40580.513432,36542.117228
4,21645.866492,8143.756874,25412.344668,22872.300857
...,...,...,...,...
4995,29599.922734,11137.258549,34750.665527,31285.248877
4996,43440.963803,16344.840943,50999.190438,45918.333525
4997,28102.664779,10574.433330,32992.726219,29707.366276
4998,27110.204514,10201.654362,31828.354699,28656.813893


In [19]:
encryptor.get_encrypt_matrix()

array([[0.21974032, 0.51943398, 0.50929578, 0.16595714],
       [0.73401819, 0.20145713, 0.88959769, 0.08139564],
       [0.82854767, 0.31178531, 0.97267918, 0.87623969],
       [0.109165  , 0.44105726, 0.5550065 , 0.60649281]])

In [20]:
encryptor.get_decrypt_matrix()

array([[ 2.72597509, -1.73625286,  2.58102027, -4.24187058],
       [ 3.99066664, -1.95184762,  0.96357009, -2.22216333],
       [-3.10228144,  3.09979741, -2.44602429,  3.96680601],
       [-0.55385187, -1.10470084,  1.07307576,  0.39829314]])

In [21]:
encryptor.transform(X) @ encryptor.get_decrypt_matrix()

,0,1,2,3
0,1.000000e+00,41.0,49600.0,1.000000e+00
1,-1.109554e-11,46.0,38000.0,1.000000e+00
2,1.782111e-13,29.0,21000.0,1.854575e-11
3,-2.599650e-12,21.0,41700.0,2.000000e+00
4,1.000000e+00,28.0,26100.0,2.282261e-11
...,...,...,...,...
4995,-1.267589e-11,28.0,35700.0,2.000000e+00
4996,1.220474e-11,34.0,52400.0,1.000000e+00
4997,-2.139138e-11,20.0,33900.0,2.000000e+00
4998,1.000000e+00,22.0,32700.0,3.000000e+00


In [22]:
encryptor.fit_transform(X)

,0,1,2,3
0,41126.388189,15473.771703,48282.425209,43465.598129
1,31518.685542,11857.549937,37003.285397,33301.458785
2,17420.787642,6553.333806,20452.061144,18403.393893
3,34566.070640,13006.560220,40580.513432,36542.117228
4,21645.866492,8143.756874,25412.344668,22872.300857
...,...,...,...,...
4995,29599.922734,11137.258549,34750.665527,31285.248877
4996,43440.963803,16344.840943,50999.190438,45918.333525
4997,28102.664779,10574.433330,32992.726219,29707.366276
4998,27110.204514,10201.654362,31828.354699,28656.813893


**Обоснование**

Как мы уже показали выше, качество линейной регрессии рпи умножении на обратимую матрицу не меняется, поэтому в классе NumericFeaturesEncrypter просто умножим X на согласованную с ней по размеру квадратную матрицу из случайных значений в диапазоне [0,1).

## Проверка алгоритма

### Разделение на обучающую/тестовую выборки

Разобьем данные на обучающую и тестовые выборки, на вход подадим X, преобразованное X (с помощью  `encryptor.fit_transform()` ) и y:

In [23]:
(X_train, X_test, X_train_c, X_test_c, y_train, y_test) = train_test_split(
    X,
    encryptor.fit_transform(X),
    y,
    test_size=0.25,
    random_state=5_12_22,
)

### DummyRegressor

Для проверки результата на адекватность посмотрим на качество модели DummyRegressor:

In [24]:
dummy = DummyRegressor()
dummy.fit(X_train, y_train)
print("R2 score:", dummy.score(X_test, y_test))

R2 score: -0.0006523974933918542


In [25]:
dummy = DummyRegressor()
dummy.fit(X_train_c, y_train)
print("R2 score:", dummy.score(X_test_c, y_test))

R2 score: -0.0006523974933918542


### Класс линейной регрессии

Напишем функцию для расчета MSE:

In [26]:
def MSE(y_hat, y):
    return ((y_hat - y) ** 2).sum() / len(y)

Напишем наш собственный класс линейной регрессии с минимизацией функции потерь MSE. Для вычисления коэффициентов $w$ будем использовать формулу:

$$
w = (X^T X)^{-1} X^T y
$$

, где считается обратная матрица для квадратной, в таком случае мы можем использовать функцию

```python 
np.linalg.inv(...)
```

, иначе нам пришлось бы получать псевдообратную матрицу для прямоугольной матрицы $ X $ в выражении $w =  X^{-1} y$

В качестве score в нашем классе будем использовать коэффициент детерминации:

$$
R2 = 1- \frac{MSE(model)}{MSE(mean)}
$$

In [27]:
class CustomLinearRegressionLossMSE(BaseEstimator):
    def __init__(self):
        pass

    def fit(self, X, y):
        # adding ones to X as first column - for w0
        X_ext = np.column_stack((np.ones(len(X)), X))
        self.w = (np.linalg.inv(X_ext.T @ X_ext) @ X_ext.T @ y).ravel()
        return self

    def predict(self, X):
        return self.w[1:] @ X.T + self.w[0]
        # return self.w @ np.column_stack((np.ones(len(X)), X)).T

    def decision_function(self, X):
        return self.w

    def score(self, X, y):
        return 1 - MSE(self.predict(X), y) / MSE([y.mean()] * len(y), y)

### Результаты исходной задачи

In [28]:
linear_regr = CustomLinearRegressionLossMSE()
linear_regr.fit(X_train, y_train)
w = linear_regr.decision_function(X_train)
print("Coefficients of linear regression:\n", w)
print("MSE of model:", MSE(linear_regr.predict(X_train), y_train))
print("R2 score:", linear_regr.score(X_test, y_test))

Coefficients of linear regression:
 [-9.31393867e-01  5.76950437e-03  3.57076407e-02 -3.52913598e-07
 -9.90289607e-03]
MSE of model: 0.12294534111615707
R2 score: 0.4092244127922975


### Результаты зашифрованной задачи

In [29]:
linear_regr_c = CustomLinearRegressionLossMSE()
linear_regr_c.fit(X_train_c, y_train)
w_c = linear_regr_c.decision_function(X_train_c)
print("Coefficients of linear regression:\n", w_c)
print("MSE of model:", MSE(linear_regr_c.predict(X_train_c), y_train))
print("R2 score:", linear_regr_c.score(X_test_c, y_test))

Coefficients of linear regression:
 [-0.93139612 -0.00426288 -0.02466476  0.05350453 -0.04658655]
MSE of model: 0.12294534111648699
R2 score: 0.4092244783008542


Видим, что MSE исходной задачи и преобразованной равны вплоть до 13 знака после запятой. Погрешность такого порядка, как мы уже видели, возникает при умножении матриц из чисел с плавающей запятой.

Кроме того, видим, что коэф. детерминации R2 получился одинаковый до 11 знака после запятой. Опять же погрешность здесь могла вырасти относительно MSE из-за более сложной формулы для R2.

Проверим также следующую формулу связи коэффициентов:

$$
w_c = P^{-1}w
$$


Для этого добавим в матрицу-дешифровщик первую строку и столбец с элементами-нулями кроме самого первого элемента, чтобы при умножении получалось $w_0$:

In [30]:
print(w_c)
print(
    np.row_stack(
        (
            np.array([1, 0, 0, 0, 0]),
            np.column_stack((np.array([0, 0, 0, 0]), encryptor.get_decrypt_matrix())),
        )
    )
    @ w
)

[-0.93139612 -0.00426288 -0.02466476  0.05350453 -0.04658655]
[-0.93139387 -0.00426408 -0.02466619  0.05350582 -0.04658635]


Также получили одинаковые с учетом погрешности результаты.

### Pipeline

Так же проверим наши результы с помощью Pipeline, воспользовавшись нашими кастомными классами и добавим масштабирование и центрирование признаков с помощью `StandardScaler()` :

In [31]:
pipeline_with_encrypt = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("encrypt", NumericFeaturesEncrypter()),
        ("model", CustomLinearRegressionLossMSE()),
    ]
)

pipeline_without_encrypt = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("model", CustomLinearRegressionLossMSE()),
    ]
)

In [32]:
pipeline_with_encrypt.fit(X_train, y_train)
pipeline_without_encrypt.fit(X_train, y_train)

print(
    "Coefficients of linear regression for encrypted data:\n",
    pipeline_with_encrypt.decision_function(X_train),
)
print(
    "Coefficients of linear regression for not encrypted data:\n",
    pipeline_without_encrypt.decision_function(X_train),
)

Coefficients of linear regression for encrypted data:
 [ 0.15093333 -0.4834025  -0.56134947  0.89934054 -0.34550297]
Coefficients of linear regression for not encrypted data:
 [ 0.15093333  0.00288465  0.30401476 -0.00351612 -0.01076311]


Проверим что обе моделей выдают схожие результаты предсказаний с помощью `np.allclose`:

In [33]:
print(
    "All elements are close:",
    np.allclose(
        pipeline_with_encrypt.predict(X_test),
        pipeline_without_encrypt.predict(X_test),
    ),
)
print(pipeline_with_encrypt.predict(X_test))
print(pipeline_without_encrypt.predict(X_test))

All elements are close: True
[ 0.49421506  0.17471674 -0.05257235 ...  0.3789891   0.63243643
 -0.2383143 ]
[ 0.49421506  0.17471674 -0.05257235 ...  0.3789891   0.63243643
 -0.2383143 ]


In [34]:
print("R2 score encrypted:", pipeline_with_encrypt.score(X_test, y_test))
print("R2 score no encryption:", pipeline_without_encrypt.score(X_test, y_test))
print(
    "R2 scores are same: ",
    pipeline_with_encrypt.score(X_test, y_test)
    == pipeline_without_encrypt.score(X_test, y_test),
)

R2 score encrypted: 0.4092244127922975
R2 score no encryption: 0.4092244127922975
R2 scores are same:  True


Благодаря масштабированию здесь уже получили полностью идентичные метрики качества для исходных и преобразованных признаков.

### Pipeline для LinearRegression из sklearn

Также проверим на модели LinearRegression из sklearn:

In [35]:
pipeline_with_encrypt = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("encrypt", NumericFeaturesEncrypter()),
        ("model", LinearRegression()),
    ]
)

pipeline_without_encrypt = Pipeline(
    [
        ("scaler", StandardScaler()),
        ("model", LinearRegression()),
    ]
)

In [36]:
pipeline_with_encrypt.fit(X_train, y_train)
pipeline_without_encrypt.fit(X_train, y_train)
print(
    "Coefficients of linear regression for encrypted data:\n",
    pipeline_with_encrypt.named_steps["model"].coef_,
)
print(
    "Coefficients of linear regression for not encrypted data:\n",
    pipeline_without_encrypt.named_steps["model"].coef_,
)

Coefficients of linear regression for encrypted data:
 [-0.4834025  -0.56134947  0.89934054 -0.34550297]
Coefficients of linear regression for not encrypted data:
 [ 0.00288465  0.30401476 -0.00351612 -0.01076311]


Проверим что обе моделей выдают схожие результаты предсказаний с помощью `np.allclose`:

In [37]:
print(
    "All elements are close:",
    np.allclose(
        pipeline_with_encrypt.predict(X_test),
        pipeline_without_encrypt.predict(X_test),
    ),
)
print(pipeline_with_encrypt.predict(X_test))
print(pipeline_without_encrypt.predict(X_test))

All elements are close: True
[ 0.49421506  0.17471674 -0.05257235 ...  0.3789891   0.63243643
 -0.2383143 ]
[ 0.49421506  0.17471674 -0.05257235 ...  0.3789891   0.63243643
 -0.2383143 ]


In [38]:
print("R2 score encrypted:", pipeline_with_encrypt.score(X_test, y_test))
print("R2 score no encryption:", pipeline_without_encrypt.score(X_test, y_test))
print(
    "R2 scores are same: ",
    pipeline_with_encrypt.score(X_test, y_test)
    == pipeline_without_encrypt.score(X_test, y_test),
)

R2 score encrypted: 0.4092244127922975
R2 score no encryption: 0.4092244127922975
R2 scores are same:  True


## Выводы

В данной работе мы нашли способ защитить численные персональные данные путем умножения признаков на обратимую матрицу. Мы показали, что функция потерь при таком преобразовании признаков не меняется, соответственно качество модели остается тем же. Мы также подтвердили это экспериментально различными способами, измеряя метрику коэффициента детериминации R2. Кроме того, мы, ожидаемо, получили серьезное увеличение R2-метрики по сравению с dummy моделью.